In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from aurora import AuroraSmall
from aurora.normalisation import locations, scales
import xarray as xr
from aurora import  Batch, Metadata, rollout
from aurora.normalisation import locations, scales
import torch
import numpy as np
import torch.optim as optim
from scipy.ndimage import zoom
locations["aco2rec"] = 0.0
scales["aco2rec"] = 1.0
locations["aco2gpp"] = 0.0
scales["aco2gpp"] = 1.0
locations["aco2nee"] = 0.0
scales["aco2nee"] = 1.0
locations["fco2rec"] = 0.0
scales["fco2rec"] = 1.0
locations["fco2gpp"] = 0.0
scales["fco2gpp"] = 1.0
locations["fco2nee"] = 0.0
scales["fco2nee"] = 1.0
locations["a1_1000"] = 0.0
scales["a1_1000"] = 1.0
# Load the Aurora model
device_ids = [ 1,2,3]
model =AuroraSmall(
    use_lora=False,
    autocast=True,
    surf_vars=(
        "aco2rec",
        "aco2gpp",
        "aco2nee",
        "fco2rec",
        "fco2gpp",
        "fco2nee",),
    static_vars=("z","lsm"),
    atmos_vars=("t"),
).cuda()
optimizer = optim.SGD(model.parameters(), lr=0.01)
# model.load_checkpoint("microsoft/aurora", r"C:\Users\DC\Documents\vscode4\aurora_co2_2\aurora-0.25-finetuned.ckpt", strict=False)
# File paths
co2_data_sfc = r"C:\Users\DC\Documents\vscode4\7bc2ad758dd66a384b66b64346e658f3\data_sfc.nc"
download_path = r'C:\Users\DC\Documents\vscode4\ubuntu_vs\vscode\VScode2\vscode\down'

# Load datasets
co2_data_sfc = xr.open_dataset(co2_data_sfc, engine="netcdf4")
static_vars_ds = xr.open_dataset(f"{download_path}/static.nc", engine="netcdf4")
surf_vars_ds = xr.open_dataset(f"{download_path}/2023-01-01-surface-level.nc", engine="netcdf4")

atmos_vars_ds = xr.open_dataset(f"{download_path}/2023-01-01-atmospheric.nc", engine="netcdf4")
loss_fn1 = torch.nn.MSELoss(reduction='none')

In [ ]:
'''
Author: DC
Date: 2024-11-06 21:28:56
LastEditTime: 2024-11-07 11:01:40
LastEditors: DC
Description: 
FilePath: \aurora_co2_2\finetune3.ipynb
Never lose my passion
'''
for epoch in range(100):
    for i in range(0, 22):
        for j in range(1, 42):
            var_names = ['aco2rec', 'aco2gpp', 'aco2nee', 'fco2rec', 'fco2gpp', 'fco2nee']
            var_dict = {}
            for var in var_names:
                arr = zoom(co2_data_sfc[var][:, i, :, :][[j-1, j]], (1, 720 / co2_data_sfc[var].shape[2], 1440 / co2_data_sfc[var].shape[3]), order=1)
                var_dict[var] = arr[None]

            arr_aco2rec = var_dict['aco2rec']
            arr_aco2gpp = var_dict['aco2gpp']
            arr_aco2nee = var_dict['aco2nee']
            arr_fco2rec = var_dict['fco2rec']
            arr_fco2gpp = var_dict['fco2gpp']
            arr_fco2nee = var_dict['fco2nee']

            arr_np = np.ones((1, 2, 13, 720, 1440))
            arr_np2 = np.ones((720, 1440))

            batch = Batch(
                surf_vars={
                    "aco2rec": torch.from_numpy(arr_aco2rec),
                    "aco2gpp": torch.from_numpy(arr_aco2gpp),
                    "aco2nee": torch.from_numpy(arr_aco2nee),
                    "fco2rec": torch.from_numpy(arr_fco2rec),
                    "fco2gpp": torch.from_numpy(arr_fco2gpp),
                    "fco2nee": torch.from_numpy(arr_fco2nee),
                },
                static_vars={
                    "z": torch.from_numpy(arr_np2),
                    "lsm": torch.from_numpy(arr_np2),
                },
                atmos_vars={
                    "t": torch.from_numpy(arr_np),
                },
                metadata=Metadata(
                    lat=torch.from_numpy(atmos_vars_ds.latitude[:720].values),
                    lon=torch.from_numpy(atmos_vars_ds.longitude[:1440].values),
                    time=(co2_data_sfc['valid_time'].values.astype("datetime64[s]").tolist()[0][i],),
                    atmos_levels=tuple(int(level) for level in atmos_vars_ds.pressure_level.values),
                ),
            )
            batch=batch.to('cuda')

            loss = 0
            if j != 1:
                # former_prerds=former_prerds.to('cpu')
               
                for key in preds[0].surf_vars.keys():
                    
                    print(former_prerds[0].surf_vars[key].shape)
                    print("11111111")
                    print(batch.surf_vars[key].shape)
                    print()
                    pred1=former_prerds[0].surf_vars[key]
                    # true1=batch.surf_vars[key][:,0,:,:]
                    true1=torch.from_numpy(var_dict[key][:,0,:,:]).to('cuda')
                    pred2=former_prerds[1].surf_vars[key].to('cpu')
                    true2=torch.from_numpy(var_dict[key][:,1,:,:]).to('cuda')
                    print(pred1.device)
                    print(true1.device)

                    loss += loss_fn1(pred1, true1)
                    # loss += loss_fn1(pred2, true2)
                    # loss=loss.to('cuda')
                print("loss")
                print(loss.sum())
                print(loss.shape)
                print()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # del former_prerds,loss,pred1,true1,pred2,true2
                # torch.cuda.empty_cache()
                # batch.to('cuda')


            model = model.cuda()
            model.train()
            
            preds = [pred for pred in rollout(model, batch, steps=2)]
            former_prerds = preds

            # print()
